In [1]:
#Import Packages
import numpy as np
import os
import pickle
from multicontact_learning_local_objectives.python.machine_learning.ml_utils import *
import matplotlib.pyplot as plt #Matplotlib

In [2]:
#Define Path for Storing Trajectories
#Collect Data Points Path
TrajPath = "/home/jiayu/Desktop/multicontact_learning_local_objectives/data/antfarm_2steps/"

In [3]:
#Shift world frame type: StanceFoot, (Abandon) InitCoM,  (Abandon) InitSurfBorder, None
Shift_World_Frame_Type = "StanceFoot";       print("Shift World Frame to:\n", Shift_World_Frame_Type)
#Contact Location Representation Type 1) 3DPoints 2) ConvexCombination 3) FollowRectangelBorder
Contact_Representation_Type = "FollowRectangelBorder";   print("Contact Location Representation Type:\n ", Contact_Representation_Type)
#Scaling Vectors? 1 menas no scaling (Now scaling does not make much diff)
ScaleFactor = 1;      print("Scaling Factor of all quantities (Except Left and Right Swing Flag):\n", ScaleFactor)

Shift World Frame to:
 StanceFoot
Contact Location Representation Type:
  FollowRectangelBorder
Scaling Factor of all quantities (Except Left and Right Swing Flag):
 1


In [4]:
#Start Collecting Data Points

#get all the file names
filenames = os.listdir(TrajPath)

#loop all the files
for filename in filenames:
    if ".p" in filename:#a data file
        print("Process: ",filename)

        #Load data
        with open(TrajPath+filename, 'rb') as f:
            data= pickle.load(f)

            #for roundNum in range(1):
            for roundNum in range(data["Num_of_Rounds"]):
                print("   Process Round: ", roundNum)
                #Get Single Optimization Result of current result
                singleOptResult = data["SingleOptResultSavings"][roundNum]
                
                #Get data Points
                xW, yW = getDataPoints(SingleOptRes = singleOptResult, Shift_World_Frame = None, ContactRepresentationType = Contact_Representation_Type, VectorScaleFactor = ScaleFactor) #InitCoM; InitSurfBorder

                xL, yL = getDataPoints(SingleOptRes = singleOptResult, Shift_World_Frame = Shift_World_Frame_Type, ContactRepresentationType = Contact_Representation_Type, VectorScaleFactor = ScaleFactor) #InitCoM; InitSurfBorder

                initConfig_W, terminalConfig_W = getInitTerminalConfig(SingleOptRes = singleOptResult, Shift_World_Frame = None)

                rotateTran, homoTran = Compute_Local_to_World_Frame_Transformation(InitConfig = initConfig_W, Local_Frame_Name = Shift_World_Frame_Type)
                
                #print("Homo Tran")
                #print(homoTran)

                if np.sum(np.hstack((np.array([xL[0:3]]),np.array([[1.0]])))-(np.linalg.inv(homoTran)@np.hstack((np.array([xW[0:3]]),np.array([[1.0]]))).T).T) > 0.0:
                    print("Init CoM World: ", np.hstack((np.array([xW[0:3]]),np.array([[1.0]]))))
                    print("Init CoM Local: ", np.hstack((np.array([xL[0:3]]),np.array([[1.0]]))))
                    print("CoM from World Frame to Local:\n",(np.linalg.inv(homoTran)@np.hstack((np.array([xW[0:3]]),np.array([[1.0]]))).T).T)
                    print("diff: ", np.sum(np.hstack((np.array([xL[0:3]]),np.array([[1.0]])))-(np.linalg.inv(homoTran)@np.hstack((np.array([xW[0:3]]),np.array([[1.0]]))).T).T))
                    print(' ')

                if np.sum(np.hstack((np.array([xL[3:6]]),np.array([[1.0]])))-(np.linalg.inv(rotateTran)@np.hstack((np.array([xW[3:6]]),np.array([[1.0]]))).T).T) > 0.0:
                    print("Init CoMdot World: ", np.hstack((np.array([xW[3:6]]),np.array([[1.0]]))))
                    print("Init CoMdot Local: ", np.hstack((np.array([xL[3:6]]),np.array([[1.0]]))))
                    print("CoMdot from World Frame to Local:\n",(np.linalg.inv(rotateTran)@np.hstack((np.array([xW[3:6]]),np.array([[1.0]]))).T).T)
                    print("diff: ", np.sum(np.hstack((np.array([xL[3:6]]),np.array([[1.0]])))-(np.linalg.inv(rotateTran)@np.hstack((np.array([xW[3:6]]),np.array([[1.0]]))).T).T))
                    print(' ')

                if np.sum(np.hstack((np.array([xL[6:9]]),np.array([[1.0]])))-(np.linalg.inv(rotateTran)@np.hstack((np.array([xW[6:9]]),np.array([[1.0]]))).T).T) > 0.0:
                    print("Init L World: ", np.hstack((np.array([xW[6:9]]),np.array([[1.0]]))))
                    print("Init L Local: ", np.hstack((np.array([xL[6:9]]),np.array([[1.0]]))))
                    print("L from World Frame to Local:\n",(np.linalg.inv(rotateTran)@np.hstack((np.array([xW[3:6]]),np.array([[1.0]]))).T).T)
                    print("diff: ", np.sum(np.hstack((np.array([xL[6:9]]),np.array([[1.0]])))-(np.linalg.inv(rotateTran)@np.hstack((np.array([xW[6:9]]),np.array([[1.0]]))).T).T))
                    print(' ')

                if xW[15] == 1:
                   diffs = np.sum(np.hstack((np.array([xL[9:12]]),np.array([[1.0]])))-(np.linalg.inv(homoTran)@np.hstack((np.array([xW[9:12]]),np.array([[1.0]]))).T).T) 
                else: 
                   diffs = np.sum(np.hstack((np.array([xL[9:12]]),np.array([[1.0]])))-(np.linalg.inv(homoTran)@np.hstack((np.array([xW[12:15]]),np.array([[1.0]]))).T).T)
                if diffs > 0.0:
                    print("Init Swing World: ", np.hstack((np.array([xW[9:12]]),np.array([[1.0]])))) if xW[15] == 1 else print("Init Swing World: ", np.hstack((np.array([xW[12:15]]),np.array([[1.0]]))))
                    print("Init Swing Local: ", np.hstack((np.array([xL[9:12]]),np.array([[1.0]]))))
                    print("Init Swing from World Frame to Local:\n",(np.linalg.inv(homoTran)@np.hstack((np.array([xW[9:12]]),np.array([[1.0]]))).T).T) if xW[15] == 1 else print("Init Swing from World Frame to Local:\n",(np.linalg.inv(homoTran)@np.hstack((np.array([xW[12:15]]),np.array([[1.0]]))).T).T)
                    print("diff: ", np.sum(np.hstack((np.array([xL[9:12]]),np.array([[1.0]])))-(np.linalg.inv(homoTran)@np.hstack((np.array([xW[9:12]]),np.array([[1.0]]))).T).T)) if xW[15] == 1 else print("diff: ", np.sum(np.hstack((np.array([xL[9:12]]),np.array([[1.0]])))-(np.linalg.inv(homoTran)@np.hstack((np.array([xW[12:15]]),np.array([[1.0]]))).T).T))
                    print(' ')

                for surfvertex_cnt in range(16):
                    if np.sum(np.hstack((np.array([xL[13+3*surfvertex_cnt:13+3*(surfvertex_cnt+1)]]),np.array([[1.0]])))-(np.linalg.inv(homoTran)@np.hstack((np.array([xW[16+3*surfvertex_cnt:16+3*(surfvertex_cnt+1)]]),np.array([[1.0]]))).T).T) >0.0:
                        print("Init Vertex" + str(surfvertex_cnt) + " World: ", np.hstack((np.array([xW[16+3*surfvertex_cnt:16+3*(surfvertex_cnt+1)]]),np.array([[1.0]]))))
                        print("Init Vertex" + str(surfvertex_cnt) + " Local: ", np.hstack((np.array([xL[13+3*surfvertex_cnt:13+3*(surfvertex_cnt+1)]]),np.array([[1.0]]))))
                        print("Init Vertex" + str(surfvertex_cnt) + "from World Frame to Local:\n",(np.linalg.inv(homoTran)@np.hstack((np.array([xW[16+3*surfvertex_cnt:16+3*(surfvertex_cnt+1)]]),np.array([[1.0]]))).T).T)
                        print("diff: ", np.sum(np.hstack((np.array([xL[13+3*surfvertex_cnt:13+3*(surfvertex_cnt+1)]]),np.array([[1.0]])))-(np.linalg.inv(homoTran)@np.hstack((np.array([xW[16+3*surfvertex_cnt:16+3*(surfvertex_cnt+1)]]),np.array([[1.0]]))).T).T))
                        print(' ')

                if np.sum(np.hstack((np.array([yL[0:3]]),np.array([[1.0]])))-(np.linalg.inv(homoTran)@np.hstack((np.array([yW[0:3]]),np.array([[1.0]]))).T).T) > 0.0:
                    print("LocalObj CoM World: ", np.hstack((np.array([yW[0:3]]),np.array([[1.0]]))))
                    print("LocalObj CoM Local: ", np.hstack((np.array([yL[0:3]]),np.array([[1.0]]))))
                    print("Local Obj CoM from World Frame to Local:\n",(np.linalg.inv(homoTran)@np.hstack((np.array([yW[0:3]]),np.array([[1.0]]))).T).T)
                    print("diff: ", np.sum(np.hstack((np.array([yL[0:3]]),np.array([[1.0]])))-(np.linalg.inv(homoTran)@np.hstack((np.array([yW[0:3]]),np.array([[1.0]]))).T).T))
                    print(' ')
                
                if np.sum(np.hstack((np.array([yL[3:6]]),np.array([[1.0]])))-(np.linalg.inv(rotateTran)@np.hstack((np.array([yW[3:6]]),np.array([[1.0]]))).T).T) > 0.0:
                    print("LocalObj CoMdot World: ", np.hstack((np.array([yW[3:6]]),np.array([[1.0]]))))
                    print("LocalObj CoMdot Local: ", np.hstack((np.array([yL[3:6]]),np.array([[1.0]]))))
                    print("Loal Obj CoMdot from World Frame to Local:\n",(np.linalg.inv(rotateTran)@np.hstack((np.array([yW[3:6]]),np.array([[1.0]]))).T).T)
                    print("diff: ", np.sum(np.hstack((np.array([yL[3:6]]),np.array([[1.0]])))-(np.linalg.inv(rotateTran)@np.hstack((np.array([yW[3:6]]),np.array([[1.0]]))).T).T))
                    print(' ')

                if np.sum(np.hstack((np.array([yL[6:9]]),np.array([[1.0]])))-(np.linalg.inv(rotateTran)@np.hstack((np.array([yW[6:9]]),np.array([[1.0]]))).T).T) > 0.0:
                    print("Local Obj L World: ", np.hstack((np.array([yW[6:9]]),np.array([[1.0]]))))
                    print("Local Obj L Local: ", np.hstack((np.array([yL[6:9]]),np.array([[1.0]]))))
                    print("Local Obj L from World Frame to Local:\n",(np.linalg.inv(rotateTran)@np.hstack((np.array([yW[3:6]]),np.array([[1.0]]))).T).T)
                    print("diff: ", np.sum(np.hstack((np.array([yL[6:9]]),np.array([[1.0]])))-(np.linalg.inv(rotateTran)@np.hstack((np.array([yW[6:9]]),np.array([[1.0]]))).T).T))
                    print(' ')

Process:  16degree.p
   Process Round:  0
   Process Round:  1
   Process Round:  2
   Process Round:  3
   Process Round:  4
   Process Round:  5
   Process Round:  6
   Process Round:  7
   Process Round:  8
   Process Round:  9
   Process Round:  10
   Process Round:  11
   Process Round:  12
   Process Round:  13
   Process Round:  14
   Process Round:  15
   Process Round:  16
   Process Round:  17
   Process Round:  18
   Process Round:  19
Process:  12degree.p
   Process Round:  0
   Process Round:  1
   Process Round:  2
   Process Round:  3
   Process Round:  4
   Process Round:  5
   Process Round:  6
   Process Round:  7
   Process Round:  8
   Process Round:  9
   Process Round:  10
   Process Round:  11
   Process Round:  12
   Process Round:  13
   Process Round:  14
   Process Round:  15
   Process Round:  16
   Process Round:  17
   Process Round:  18
   Process Round:  19
Process:  8degree.p
   Process Round:  0
   Process Round:  1
   Process Round:  2
   Process Roun